<a href="https://colab.research.google.com/github/JSJeong-me/Prompt-Engineering/blob/main/2-1-Cat-Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# 사전 학습된 VGG16 모델 로드
model = models.vgg16(pretrained=True)

# VGG16의 마지막 분류기 레이어 (classifier) 수정
# 기본적으로 VGG16의 classifier는 (4096 -> 4096 -> 1000) 형태로 되어 있습니다.
# 이를 (4096 -> 4096 -> 2)로 수정합니다.

model.classifier[6] = nn.Linear(4096, 2)

# 모델 수정 후 출력 확인
print(model)

In [ ]:


# 예시 입력 데이터
input_data = torch.randn(1, 3, 224, 224)

# 모델을 통해 예측값 출력
output = model(input_data)
print(output)


In [ ]:
!wget https://github.com/JSJeong-me/Generate_AI_for_Image/raw/main/images/cat1.png

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO

# 이미지 다운로드 (cat1.png 파일을 다운로드)
# !wget https://github.com/JSJeong-me/Generate_AI_for_Image/raw/main/images/cat1.png -O cat1.png

# 이미지 불러오기 및 RGB로 변환
image = Image.open('cat1.png').convert('RGB')

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 이미지 텐서로 변환
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)  # 배치 차원 추가

# GPU가 사용 가능하다면, 모델과 데이터를 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 사전 학습된 VGG16 모델 불러오기
model = models.vgg16(pretrained=True)

# 모델의 마지막 레이어를 고양이와 개를 구별하도록 수정
model.classifier[6] = nn.Linear(4096, 2)

# 모델을 평가 모드로 설정
model.eval()

# 모델을 장치로 이동
model.to(device)
input_batch = input_batch.to(device)

# 예측
with torch.no_grad():
    output = model(input_batch)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

# 결과 출력
categories = ['cat', 'dog']
_, predicted_idx = torch.max(probabilities, 0)

print(f"Predicted: {categories[predicted_idx]}")
